## Tiling

In [ ]:
import os
from google.colab import drive, files
import yaml

drive.mount('/content/drive')

In [ ]:
# dependencies
!apt-get install -y openslide-tools
!pip install openslide-python wandb
!pip install torch torchvision torchaudio tqdm

In [ ]:
# clone repo
!git clone https://github.com/vkola-lab/tmi2022.git
%cd tmi2022

In [ ]:
!pip install pydicom

In [ ]:
import os, glob, subprocess, shlex

wsi_dir = "/content/drive/MyDrive/TCGA_WSIs"
output_patches = "/content/drive/MyDrive/TCGA_WSIs/patches"

# all SVS (recursive)
wsi_paths = glob.glob(os.path.join(wsi_dir, "**", "*.svs"), recursive=True)

def stem(p):
    return os.path.splitext(os.path.basename(p))[0]

def already_tiled(p):
    s = stem(p)
    # typical output dir from tile_WSI.py (adjust if yours differs)
    cand = os.path.join(output_patches, f"{s}_files")
    if os.path.isdir(cand):
        # optionally also ensure it has any files inside (skip half-written dirs)
        for _root, _dirs, files in os.walk(cand):
            if files:
                return True
    return False

# only the ones still missing
to_tile = [p for p in wsi_paths if not already_tiled(p)]

print(f"SVS found: {len(wsi_paths)}")
print(f"Already tiled: {len(wsi_paths) - len(to_tile)}")
print(f"To tile now: {len(to_tile)}")

for wsi_path in to_tile:
    print("Tiling", wsi_path)
    cmd = f'python src/tile_WSI.py -s 512 -e 0 -j 10 -B 50 -M 20 -o {shlex.quote(output_patches)} {shlex.quote(wsi_path)}'
    ret = subprocess.call(cmd, shell=True)
    if ret != 0:
        print("FAILED:", wsi_path)


In [ ]:
# How many slides finished tiling?
import os, glob
patch_root = "/content/drive/MyDrive/TCGA_WSIs/patches"
done = sorted(glob.glob(os.path.join(patch_root, "*_files")))
len(done)

In [ ]:
import os, glob

svs_root     = "/content/drive/MyDrive/TCGA_WSIs"        # where the .svs live (nested)
patch_root   = "/content/drive/MyDrive/TCGA_WSIs/patches" # where tile folders are written

# all slides (recursive)
all_svs = sorted(glob.glob(os.path.join(svs_root, "**", "*.svs"), recursive=True))
svs_base = {os.path.splitext(os.path.basename(p))[0] for p in all_svs}

# all produced tile folders like <basename>_files
tile_dirs = sorted(glob.glob(os.path.join(patch_root, "*_files")))
tiled_base = {os.path.basename(d).replace("_files","") for d in tile_dirs}

missing = sorted(svs_base - tiled_base)
print(f"SVS found: {len(all_svs)}")
print(f"Tile folders: {len(tile_dirs)}")
print(f"Missing (no *_files created): {len(missing)}")

In [ ]:
from openslide import OpenSlide
bad = []
for p in all_svs:
    try:
        _ = OpenSlide(p)
    except Exception as e:
        bad.append((os.path.basename(p), str(e)))
print("Unreadable slides:", len(bad))

In [ ]:
wsi_dir = "/content/drive/MyDrive/TCGA_WSIs"
output_patches = "/content/drive/MyDrive/TCGA_WSIs/patches"

os.listdir(output_patches)

In [ ]:
import glob
import pandas as pd
%cd /content/tmi2022/feature_extractor

# Adjust path if needed
#image_paths = glob.glob("/content/drive/MyDrive/TCGA_WSIs/patches/*/*/*.jpeg", recursive=True)

# Convert to DataFrame
#df = pd.DataFrame(image_paths)

# Save to CSV
#df.to_csv("patches.csv", index=False, header=False)

!find /content/drive/MyDrive/TCGA_WSIs/patches -type f -iname '*.jpeg' > patches.csv


In [ ]:
import yaml

%cd /content/tmi2022

cfg_path = "/content/tmi2022/feature_extractor/config.yaml"
config = {
    'batch_size': 64,
    'epochs': 20,
    'eval_every_n_epochs': 1,
    'fine_tune_from': '',
    'log_every_n_steps': 25,
    'weight_decay': "1e-5",
    'fp16_precision': False,
    'n_gpu': 1,
    'gpu_ids': [0],
    'model': {
        'out_dim': 512,
        'base_model': "resnet50"
    },
    'dataset': {
        's': 1,
        'input_shape': "(224, 224, 3)",   # ← keep as STRING for their eval(...)
        'num_workers': 10,
        'valid_size': 0.1
    },
    'loss': {
        'temperature': 0.5,
        'use_cosine_similarity': True
    }
    # you can also add 'lr': "1e-5" if somewhere else they eval lr from config
}

with open(cfg_path, 'w') as f:
    yaml.dump(config, f, sort_keys=False)

print(open(cfg_path).read())


In [ ]:
import yaml, pathlib
cfg_path = pathlib.Path("/content/tmi2022/feature_extractor/config.yaml")
cfg = yaml.safe_load(open(cfg_path))

cfg['batch_size'] = 64          # try 64; if OOM persists, try 32
cfg['dataset']['input_shape'] = "(224, 224, 3)"  # keep as string for their eval()
cfg['dataset']['num_workers'] = 4                # lighter on RAM
cfg['fp16_precision'] = True                     # we'll enable native AMP below
cfg['weight_decay'] = "1e-5"                     # keep as string due to eval() in repo

yaml.dump(cfg, open(cfg_path, "w"), sort_keys=False)
print(open(cfg_path).read())


In [ ]:
import os, imghdr, time
import pandas as pd
from PIL import Image, ImageFile
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

%cd /content/tmi2022/feature_extractor

# --- settings ---
CSV_IN   = "patches.csv"
CSV_OUT  = "all_patches.csv"
CHECKPOINT_EVERY = 10000   # write partial results every N files
MAX_WORKERS = 16           # Colab often benefits from 8-16 for IO-bound tasks
MIN_BYTES = 1024           # skip tiny files
EXT_OK = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".webp"}

# Make PIL less picky about large images; allow truncated loads
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

# --- load & clean paths ---
df = pd.read_csv(CSV_IN, header=None)
print(f"Original entries: {len(df)}")

# Normalize strings & drop obvious dups
paths = (
    df[0].astype(str).str.strip()
      .str.replace(r"^file://", "", regex=True)
      .dropna().drop_duplicates()
      .tolist()
)
print(f"Unique paths after basic cleanup: {len(paths)}")

def cheap_prefilter(p: str) -> bool:
    """Fast checks before touching PIL."""
    try:
        if not os.path.exists(p): return False
        if os.path.getsize(p) < MIN_BYTES: return False
        ext = os.path.splitext(p)[1].lower()
        if ext and ext not in EXT_OK: return False
        # Quick signature sniff (very fast; not perfect but good filter)
        kind = imghdr.what(p)
        if kind is None:
            # Some TIFF/WEBP may fail imghdr; keep them for PIL step if extension ok
            if ext in {".tif", ".tiff", ".webp"}:
                return True
            return False
        return True
    except Exception:
        return False

def deep_check_with_pil(p: str) -> bool:
    """Heavier verification: only run for items passing the prefilter."""
    try:
        with Image.open(p) as img:
            img.verify()  # integrity check without decoding pixels
        return True
    except Exception:
        return False

def validate_path(p: str) -> str | None:
    if not cheap_prefilter(p):
        return None
    return p if deep_check_with_pil(p) else None

valid_paths = []
start = time.time()

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = {ex.submit(validate_path, p): p for p in paths}
    for i, f in enumerate(tqdm(as_completed(futures), total=len(futures), desc="Validating")):
        res = f.result()
        if res is not None:
            valid_paths.append(res)
        # checkpoint occasionally
        if (i + 1) % CHECKPOINT_EVERY == 0:
            pd.DataFrame(valid_paths).to_csv(CSV_OUT + ".part", index=False, header=False)

elapsed = time.time() - start
print(f"Validated in {elapsed/60:.1f} min")

# Save final
df_valid = pd.DataFrame(valid_paths)
df_valid.to_csv(CSV_OUT, index=False, header=False)
print(f"Valid images: {len(df_valid)}")
print(f"Removed: {len(df) - len(df_valid)} bad entries")
print(f"Filtered CSV saved to {CSV_OUT}")


In [ ]:
%cd /content/tmi2022/feature_extractor

import yaml
from data_aug.dataset_wrapper import DataSetWrapper

cfg = yaml.load(open("config.yaml","r"), Loader=yaml.FullLoader)
train_loader, _ = DataSetWrapper(cfg['batch_size'], **cfg['dataset']).get_data_loaders()
print("Steps per epoch:", len(train_loader))


In [ ]:
# run FE
%cd /content/tmi2022/feature_extractor
!python run.py

# for pretrained model
# !wget -O model.pth https://example.com/path/to/pretrained_model.pth



In [ ]:
# plot_losses_separate.py
import os, glob
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt

# Point this to EITHER the specific run dir (that contains events.out.tfevents.*)
# OR the parent "runs" directory. The helper will auto-pick the latest event dir.
LOG_PATH = "/content/tmi2022/feature_extractor/runs"

def find_event_dir(path):
    """Return a directory that directly contains TensorBoard event files."""
    def has_events(p):
        return bool(glob.glob(os.path.join(p, "events.out.tfevents.*")))
    if os.path.isdir(path) and has_events(path):
        return path
    candidates = [d for d in glob.glob(os.path.join(path, "*"))
                  if os.path.isdir(d) and has_events(d)]
    if not candidates:
        raise SystemExit(f"No TensorBoard event files found under: {path}")
    return max(candidates, key=os.path.getmtime)

def get_series(ea, epoch_tag, step_tag):
    # prefer epoch-level; fall back to step-level
    try:
        epoch_data = ea.Scalars(epoch_tag)
    except KeyError:
        epoch_data = []
    if epoch_data:
        return [s.step for s in epoch_data], [s.value for s in epoch_data], "Epoch"
    try:
        step_data = ea.Scalars(step_tag)
    except KeyError:
        step_data = []
    return ([s.step for s in step_data], [s.value for s in step_data], "Step")

run_dir = find_event_dir(LOG_PATH)
print("Using event dir:", run_dir)

ea = EventAccumulator(run_dir)
ea.Reload()
print("Available scalar tags:", ea.Tags().get("scalars", []))

# --- fetch series ---
tr_x, tr_y, tr_xlabel = get_series(ea, "epoch_train_loss", "train_loss")
va_x, va_y, va_xlabel = get_series(ea, "epoch_val_loss", "validation_loss")

if not tr_y and not va_y:
    raise SystemExit("No train/val loss scalars found. Check tags printed above.")

# Save under run_dir/checkpoints (create if missing)
ckpt_dir = os.path.join(run_dir, "checkpoints")
os.makedirs(ckpt_dir, exist_ok=True)

# --- TRAIN plot (single series) ---
if tr_y:
    plt.figure()
    plt.plot(tr_x, tr_y)
    plt.xlabel(tr_xlabel)
    plt.ylabel("Loss")
    plt.title("SimCLR Training Loss")
    plt.grid(True)
    plt.tight_layout()
    out_train = os.path.join(ckpt_dir, "train_loss_curve.png")
    plt.savefig(out_train, dpi=200)
    plt.close()
    print("Saved:", out_train)
else:
    print("No training loss scalars found.")

# --- VALIDATION plot (single series) ---
if va_y:
    plt.figure()
    plt.plot(va_x, va_y)
    plt.xlabel(va_xlabel)
    plt.ylabel("Loss")
    plt.title("SimCLR Validation Loss")
    plt.grid(True)
    plt.tight_layout()
    out_val = os.path.join(ckpt_dir, "val_loss_curve.png")
    plt.savefig(out_val, dpi=200)
    plt.close()
    print("Saved:", out_val)
else:
    print("No validation loss scalars found.")
